# Hartree-Fock - Electron-Nuclei attraction

The Fock operator is 

$$\hat{F}(\vec{r}) = \hat{h}(\vec{r}) + \sum_{j=1}^{N} \left( \hat{J}_j(\vec{r}) - \hat{K}_j(\vec{r}) \right)$$ 

where 

$$\hat{h} = \hat{T} + \hat{V}^{(eN)}$$

In this notebook we will be working with the electron-nuclear potential matrix, namely

$$\boxed{V^{(eN)}_{ij} = \int d^3r \varphi_i^*(\vec{r}) \hat{V}^{(eN)} \varphi_j(\vec{r})}$$

In [1]:
import numpy as np
from scipy.special import erf
from Utils import * # Import functions from the previous notebook: Primitive Class, Overlap and Kinetic

Suppose we have $M$ nuclei, so 

$$\hat{V}^{(eN)} = - \sum_{J=1}^{M} \frac{Z}{r_{J}}$$

It follows that, 

\begin{align*}
\hat{V}^{(eN)}_{ij} &= \int d^3r \varphi_i(\vec{r}) \hat{V}_i^{(eN)} \varphi_j(\vec{r}) \\
&= - \int d^3r \varphi_i(\vec{r}) \sum_{J=1}^{M} \frac{Z}{r_{J}}\psi_j(\vec{r})  \\
&= -Z \sum_{\mu,\nu=1}^{K} c_{i\mu}c_{\nu j} \sum_{J=1}^{M} \underbrace{\int d^3r \phi_\mu (\vec{r}) \frac{1}{r_{J}} \phi_\nu(\vec{r})}_{Q} 
\end{align*}

The integral $Q$ is 

\begin{align*}
Q &= \int d^3r N_\mu N_\nu \frac{1}{|\vec{r}-\vec{r}_J|} \exp\left\{- \alpha_\mu (\vec{r}-\vec{r}_\mu)^2\right\} \exp\left\{- \alpha_\mu (\vec{r}-\vec{r}_\nu)^2\right\} \\
&= N_\mu N_\nu \exp\left\{- \alpha_\mu \alpha_\nu/\alpha_p (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \int d^3r \frac{1}{|\vec{r}-\vec{r}_J|} \exp\left\{- \alpha_p (\vec{r}-\vec{r}_p)^2 \right\}  
\end{align*}

where $\boxed{\alpha_p = \alpha_\mu + \alpha_\nu}$ and $\boxed{x_p = \dfrac{\alpha_\mu x_\mu + \alpha_\nu x_\nu}{\alpha_p}}$. THen, 

\begin{align*}
Q &= N_\mu N_\nu \exp\left\{- \alpha_\mu \alpha_\nu/\alpha_p (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \int d^3r \frac{1}{|\vec{r}-\vec{r}_J|} \exp\left\{- \alpha_p (\vec{r}-\vec{r}_p)^2 \right\} 
\end{align*}

Now, replacing the terms in the integral with their inverse Fourier transforms, 

$$\frac{1}{|\vec{r}-\vec{r}_J|} = \frac{4 \pi}{(2\pi)^3} \int d^3k \frac{1}{k^2} \exp\left\{ i \vec{k}\cdot(\vec{r}-\vec{r}_J) \right\} \qquad ;\qquad \exp\left\{- \alpha_p (\vec{r}-\vec{r}_p)^2 \right\}  = \frac{1}{(2\pi)^3} \left( \frac{\pi}{\alpha_p}\right)^{3/2} \int d^3k \exp\left\{ - k^2/4\alpha_p\right\} \exp\left\{ i \vec{k}\cdot(\vec{r}-\vec{r}_p) \right\}$$

it follows that,

\begin{align*}
Q &= N_\mu N_\nu \exp\left\{- \alpha_\mu \alpha_\nu/\alpha_p (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \frac{1}{(2\pi)^3} 4\pi \left( \frac{\pi}{\alpha_p}\right)^{3/2} \int d^3k \frac{1}{k^2} \exp\left\{-k^2/4\alpha_p \right\} \exp\left\{- i \vec{k}\cdot(\vec{r}_J-\vec{r}_p) \right\} \\
 &= N_\mu N_\nu \exp\left\{- \alpha_\mu \alpha_\nu/\alpha_p (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \frac{1}{(2\pi)^3} 4\pi \left( \frac{\pi}{\alpha_p}\right)^{3/2} \int_{0}^{\infty} dk \frac{1}{k^2} \exp\left\{-k^2/4\alpha_p \right\} 2\pi \int_{0}^{\pi} d\theta k^2 \sin\theta \exp\left\{- i k |\vec{r}_J-\vec{r}_p| \cos\theta \right\} \\
 &= N_\mu N_\nu \exp\left\{- \alpha_\mu \alpha_\nu/\alpha_p (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \frac{2}{\pi}  \left( \frac{\pi}{\alpha_p}\right)^{3/2} \int_{0}^{\infty} dk \exp\left\{-k^2/4\alpha_p \right\} \frac{\sin\left(k |\vec{r}_J - \vec{r}_p| \right)}{k |\vec{r}_J - \vec{r}_p|} \\
  &= N_\mu N_\nu \exp\left\{- \alpha_\mu \alpha_\nu/\alpha_p (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \frac{2}{\pi}  \left( \frac{\pi}{\alpha_p}\right)^{3/2} (\alpha_p \pi)^{1/2} \frac{1}{|\vec{r}_J - \vec{r}_p|}\int_{0}^{|\vec{r}_J-\vec{r}_p|} dy \exp\left\{-\alpha_p y^2\right\} \\
  &= N_\mu N_\nu \exp\left\{- \alpha_\mu \alpha_\nu/\alpha_p (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \frac{2\pi}{\alpha_p} \underbrace{ \frac{1}{\alpha_p^{1/2}|\vec{r}_J - \vec{r}_p|} \int_{0}^{\alpha_p^{1/2}|\vec{r}_J-\vec{r}_p|} du \exp\left\{- u^2\right\} }_{F_0\left( \alpha_p(\vec{r}_J-\vec{r}_p)^2\right)} 
\end{align*}

$$\boxed{Q = N_\mu N_\nu \exp\left\{- \alpha_\mu \alpha_\nu/\alpha_p (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \frac{2\pi}{\alpha_p} F_0\left( \alpha_p(\vec{r}_J-\vec{r}_p)^2\right)}$$

Notice that $F_0(t) = \dfrac{\sqrt{\pi}}{2} \dfrac{1}{t^{1/2}} \text{erf}(t^{1/2})$

Finally, 

\begin{align*}
\hat{V}^{(eN)}_{ij} &= -Z \sum_{\mu,\nu=1}^{K} c_{i\mu}c_{\nu j} \sum_{J=1}^{M} Q_{\mu\nu}^{J} 
\end{align*}

$$\boxed{ \hat{V}^{(eN)}_{ij} = -2 \pi Z \sum_{\mu,\nu=1}^{K} c_{i\mu}c_{\nu j} \frac{N_\mu N_\nu}{\alpha_\mu + \alpha_\nu} \exp\left\{- \frac{\alpha_\mu \alpha_\nu}{\alpha_\mu + \alpha_\nu} (\vec{r}_\mu-\vec{r}_\nu)^2 \right\} \sum_{J=1}^{M} F_0 \left( (\alpha_\mu+\alpha_\nu)\left(\vec{r}_J - \vec{r}_p\right)^2 \right) }$$

In [2]:
def F_0(t):
    if t<1e-6:
        return 1.0 - t/3.0
    else:
        return 0.5*np.sqrt(np.pi/t)*erf(np.sqrt(t))
    
def electron_nuclear_attraction(molecule, atom_coordinates, Z):
    
    n_atoms = len(Z)
    n_basis = len(molecule) # Number of atoms 
    V_en = np.zeros((n_basis,n_basis)) # Overlap between the atomic orbitals
            
    # The first loop runs over all nuclei
    for atom in range(n_atoms):
        # The second loop runs over all orbitals
        for i in range(n_basis):
            for j in range(n_basis):

                n_primitives_i = len(molecule[i]) # Number of primitives in which orbital i is decomposed
                n_primitives_j = len(molecule[j]) # Number of primitives in which orbital j is decomposed

                # The third loop runs over the primitives of the orbitals to be overlap
                for k in range(n_primitives_i): 
                    for l in range(n_primitives_j):

                        N = molecule[i][k].A * molecule[j][l].A # Product of primitive normalization constants
                        p = molecule[i][k].α + molecule[j][l].α # α_μ + α_ν
                        q = molecule[i][k].α * molecule[j][l].α / p # α_μ * α_ν / α_μ + α_ν
                        Q = molecule[i][k].coords - molecule[j][l].coords # r_μ - r_ν 
                        Q2 = np.dot(Q,Q) # (r_μ - r_ν)^2
                        
                        Rp = ( molecule[i][k].α*molecule[i][k].coords + molecule[j][l].α*molecule[j][l].coords)/p
                        RJ = atom_coordinates[atom]
                        R = RJ - Rp
                        R2 = np.dot(R,R)
                                               
                        # Electron-Nucleus potential matrix element    
                        V_en[i,j] -= (2*np.pi/p) * Z[atom] * N * molecule[i][k].coeff * molecule[j][l].coeff * np.exp(-q*Q2) * F_0(p*R2) 
                    
    return V_en

In [6]:
# STO-3G BASIS FOR 1S ORBITALS
# Primitive gaussians of the first hydrogen
H1_pgaussian1a = primitive_gaussian(0.3425250914E+01,0.1543289673E+00,[0,0,0])
H1_pgaussian1b = primitive_gaussian(0.6239137298E+00,0.5353281423E+00,[0,0,0])
H1_pgaussian1c = primitive_gaussian(0.1688554040E+00,0.4446345422E+00,[0,0,0])

# Primitive gaussians of the second hydrogen
H2_pgaussian1a = primitive_gaussian(0.3425250914E+01,0.1543289673E+00,[1.4,0,0])
H2_pgaussian1b = primitive_gaussian(0.6239137298E+00,0.5353281423E+00,[1.4,0,0])
H2_pgaussian1c = primitive_gaussian(0.1688554040E+00,0.4446345422E+00,[1.4,0,0])

# Atomic orbitals
H1_1s = [H1_pgaussian1a, H1_pgaussian1b, H1_pgaussian1c]
H2_1s = [H2_pgaussian1a, H2_pgaussian1b, H2_pgaussian1c]

# Molecule
molecule = [H1_1s, H2_1s] 
print("STO-3G for 1s orbitals in H2")

# Overlap matrix
print("\n Overlap: \n",overlap(molecule))

# Kinetic energy matrix
print("\n Kinetic Energy: \n",kinetic(molecule))

# Electron-Nuclei attraction
atom_coordinates = np.array([[0,0,0],[1.4,0,0]])
Z = [1,1]
print("\n Electron-Nuclei attraction: \n", electron_nuclear_attraction(molecule,atom_coordinates,Z))

STO-3G for 1s orbitals in H2

 Overlap: 
 [[1.         0.65931821]
 [0.65931821 1.        ]]

 Kinetic Energy: 
 [[0.76003188 0.23645466]
 [0.23645466 0.76003188]]

 Electron-Nuclei attraction: 
 [[-1.88044089 -1.19483462]
 [-1.19483462 -1.88044089]]
